In [1]:
%load_ext autoreload
%autoreload 2
import argparse
import gc
import os
import random
from typing import AnyStr
from typing import List
import ipdb
import krippendorff
from collections import defaultdict
from pathlib import Path

import numpy as np
import torch
import wandb
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torch.utils.data import random_split
from torch.optim import Adam
from tqdm import tqdm
from transformers import AdamW
from transformers import DistilBertConfig
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from datareader import *

from metrics import MultiDatasetClassificationEvaluator
from metrics import ClassificationEvaluator
from metrics import acc_f1


from metrics import plot_label_distribution
from model import MultiTransformerClassifier
from model import VanillaBert
from model import *
from sklearn.model_selection import ParameterSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

/home/jbk/miniconda3/envs/anlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bert_model = 'distilbert-base-uncased'
bert_config = DistilBertConfig.from_pretrained(bert_model, num_labels=2, output_hidden_states=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert = DistilBertForSequenceClassification(bert_config)

model = MultiViewTransformerNetworkAveragingIndividuals(
            bert_model,
            bert_config,
            4
        ).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-strea

In [3]:
model.domain_experts[0].load_state_dict(torch.load('checkpoints/model_books_3.pth'))

model.domain_experts[3].load_state_dict(torch.load('checkpoints/model_kitchen_&_housewares_3.pth'))

<All keys matched successfully>

In [4]:
tokenizer = DistilBertTokenizer.from_pretrained(bert_model)
all_dsets = [MultiDomainSentimentDataset(
    'data/sentiment-dataset',
    [domain],
    tokenizer=tokenizer,
) for domain in ["books","dvd", "electronics", "kitchen_&_housewares"]]


In [7]:
book_dset = all_dsets[0]
book_dset.set_domain_id(0)
evaluator = ClassificationEvaluator(book_dset, device, use_domain=True)
out = evaluator.evaluate(model, domain=0)
out



Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

/home/jbk/miniconda3/envs/anlp/lib/python3.9/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Evaluation: 100%|██████████| 250/250 [00:20<00:00, 12.34it/s]


((0.9735197651386261, 0.8955, 0.8734655335221907, 0.925, 0.8984944147644488),
 [])

In [9]:
evaluator = ClassificationEvaluator(book_dset, device, use_domain=True)
out = evaluator.evaluate(model, domain=2)
out

Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

/home/jbk/miniconda3/envs/anlp/lib/python3.9/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Evaluation: 100%|██████████| 250/250 [00:19<00:00, 12.56it/s]


((1.3882596783638002, 0.5165, 0.8367346938775511, 0.041, 0.07816968541468065),
 [])

In [10]:
kitchen_dset = all_dsets[3]
kitchen_dset.set_domain_id(3)
evaluator = ClassificationEvaluator(kitchen_dset, device, use_domain=True)
out = evaluator.evaluate(model, domain=0)
out

Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]/home/jbk/miniconda3/envs/anlp/lib/python3.9/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Evaluation: 100%|██████████| 250/250 [00:13<00:00, 18.93it/s]


((0.7555985431671143, 0.9895, 0.9870646766169154, 0.992, 0.9895261845386534),
 [])

In [12]:
electronics_dset = all_dsets[2]
electronics_dset.set_domain_id(2)
evaluator = ClassificationEvaluator(electronics_dset, device, use_domain=False)
out = evaluator.evaluate(model, domain=0)
out


Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]/home/jbk/miniconda3/envs/anlp/lib/python3.9/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Evaluation: 100%|██████████| 250/250 [00:15<00:00, 16.11it/s]


((0.772923944234848, 0.98, 0.9771371769383698, 0.983, 0.9800598205383848), [])

In [13]:
dvd_dset = all_dsets[1]
dvd_dset.set_domain_id(1)
evaluator = ClassificationEvaluator(dvd_dset, device, use_domain=True)
out = evaluator.evaluate(model, domain=0)
out

Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]/home/jbk/miniconda3/envs/anlp/lib/python3.9/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Evaluation: 100%|██████████| 250/250 [00:21<00:00, 11.69it/s]


((0.7760259518623353, 0.976, 0.9779116465863453, 0.974, 0.9759519038076152),
 [])